In [1]:
import pandas as pd
import numpy as np
import os
import json
import pickle
from collections import defaultdict
from datetime import datetime
import openai
import requests
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored
openai.api_key = os.environ.get('OPENAI_API_KEY')

In [2]:
user_test_path = '../beauty_data/processed_data/user_test.json'
user_valid_path = '../beauty_data/processed_data/user_valid.json'

In [3]:
with open(user_test_path, 'rb') as f:
    user_test_dict = json.load(f)
print(len(user_test_dict))

624


In [4]:
with open(user_valid_path, 'rb') as f:
    user_valid_dict = json.load(f)
print(len(user_valid_dict))

624


In [5]:
print(user_valid_dict['0'])
print(user_test_dict['0'])

[1, 2, 3, 4, 5, 5]
[2, 3, 4, 5, 5, 6]


In [6]:
item_information_path = '../beauty_data/processed_data/processed_item_information.json'
with open(item_information_path, 'r') as f:
    item_information_dict = json.load(f)
print(len(item_information_dict))

1216


In [7]:
ratings_path = '../beauty_data/processed_data/ratings.csv'
ratings_df = pd.read_csv(ratings_path)
ratings_df.head()

,item,user,rating,timestamp
0,0,0,4.0,1155081600
1,1,0,5.0,1268697600
2,2,0,5.0,1404604800
3,3,0,5.0,1404604800
4,4,0,5.0,1404604800


### Creating zeroshot data

In [8]:
def get_item_description(item_information):
    item_details = ""
    if 'title' in item_information:
        item_details += item_information['title']
    if 'brand' in item_information and item_information['brand']!='':
        item_details += f" Brand: {item_information['brand']}"
    if 'price' in item_information and item_information['price']!='':
        item_details += f" Price: {item_information['price']}"
    if 'description' in item_information and item_information['description']!='':
        item_details += f" Description: {item_information['description']}"
    return item_details

In [9]:
### Test Data
zeroshot_dict = dict()
check_missed_items = []
for user, item_list in user_test_dict.items():
    # print(user, item_list)
    user_rating_dict = ratings_df[ratings_df['user'] == int(user)]
    # print(user_rating_dict)
    prompt = "You are an expert beauty product recommender. You are provided with a user's list of recent products and their descriptions that the user purchases and whether the user liked it or disliked it. Please go through the list in order -\n"
    for item in item_list[:-1]:
        try:
            user_item_rating = user_rating_dict[user_rating_dict['item'] == item]['rating'].values[0]
        except Exception as e:
            print(f"User: {user} Item: {item} error {e}")
            continue
        if user_item_rating > 3:
            ### Liked
            prompt += f"Liked {get_item_description(item_information_dict[str(item)])}\n"
        else:
            prompt += f"Disliked {get_item_description(item_information_dict[str(item)])}\n"
    target_item = item_list[-1]
    if str(target_item) not in item_information_dict:
        continue
    target_item_details = get_item_description(item_information_dict[str(target_item)])
    prompt += f"""As an expert beauty product recommender, do the following steps -
    1. Analyze the provided list of products purchased by the user in order and summarize the user behavior by identifying the characteristics he liked and disliked about the products in at most 100 words.
    2. Explain with reasoning why the user will like or dislike the target product - {target_item_details} in atmost 100 words.
    3. Finally, from the reasoning, predict whether the user will like the target product or not in the format - Prediction: Yes or No"""
    zeroshot_dict[user] = prompt
    # break

User: 64 Item: 229 error index 0 is out of bounds for axis 0 with size 0
User: 215 Item: 399 error index 0 is out of bounds for axis 0 with size 0
User: 401 Item: 880 error index 0 is out of bounds for axis 0 with size 0
User: 427 Item: 399 error index 0 is out of bounds for axis 0 with size 0


In [10]:
zeroshot_test_path = './ctr_zeroshot_dataset/zeroshot_test.json'
with open(zeroshot_test_path, 'w+') as f:
    json.dump(zeroshot_dict, f)

In [11]:
### Valid Data
zeroshot_dict = dict()
check_missed_items = []
for user, item_list in user_valid_dict.items():
    # print(user, item_list)
    user_rating_dict = ratings_df[ratings_df['user'] == int(user)]
    # print(user_rating_dict)
    prompt = "You are an expert fashion product recommender. You are provided with a user's list of recent products and their descriptions that the user purchases and whether the user liked it or disliked it. Please go through the list in order -\n"
    for item in item_list[:-1]:
        try:
            user_item_rating = user_rating_dict[user_rating_dict['item'] == item]['rating'].values[0]
        except Exception as e:
            print(f"User: {user} Item: {item} error {e}")
            continue
        if user_item_rating > 3:
            ### Liked
            prompt += f"Liked {get_item_description(item_information_dict[str(item)])}\n"
        else:
            prompt += f"Disliked {get_item_description(item_information_dict[str(item)])}\n"
    target_item = item_list[-1]
    if str(target_item) not in item_information_dict:
        continue
    target_item_details = get_item_description(item_information_dict[str(target_item)])
    prompt += f"""As an expert fashion product recommender, do the following steps -
    1. Analyze the provided list of products purchased by the user in order and summarize the user behavior by identifying the characteristics he liked and disliked about the products in at most 100 words.
    2. Explain with reasoning why the user will like or dislike the target product - {target_item_details} in atmost 100 words.
    3. Finally, from the reasoning, predict whether the user will like the target product or not in the format - Prediction: Yes or No"""
    zeroshot_dict[user] = prompt
    # break

User: 64 Item: 229 error index 0 is out of bounds for axis 0 with size 0
User: 215 Item: 399 error index 0 is out of bounds for axis 0 with size 0
User: 401 Item: 880 error index 0 is out of bounds for axis 0 with size 0
User: 427 Item: 399 error index 0 is out of bounds for axis 0 with size 0


In [12]:
print(len(zeroshot_dict))

624


In [13]:
zeroshot_dict['34']

"You are an expert fashion product recommender. You are provided with a user's list of recent products and their descriptions that the user purchases and whether the user liked it or disliked it. Please go through the list in order -\nLiked Citre Shine Moisture Burst Shampoo - 16 fl oz Brand: Citre Shine Price: $23.00 Description: \n\nThe product is a shampoo that helps reduce psoriasis irritation and thinning hair, while also moisturizing and strengthening all hair types. However, it may not be easily found in local stores.\nLiked Bath &amp; Body Works Ile De Tahiti Moana Coconut Vanille Moana Body Wash with Tamanoi 8.5 oz Brand: Bath & Body Works Description: \n\nThe product is a high-quality, good smelling cologne for men. It comes in various scents and is infused with pheromones. However, the scent may not last long after showering.\n\nThe product is a high-quality, good smelling cologne for\nLiked Bath &amp; Body Works Ile De Tahiti Moana Coconut Vanille Moana Body Wash with Taman

In [14]:
total_len = 0
for user, prompt in zeroshot_dict.items():
    total_len += len(prompt.strip().split(' '))
print(total_len/len(zeroshot_dict))

409.59615384615387


In [15]:
users_very_long = []
for user, prompt in zeroshot_dict.items():
    if len(prompt.strip().split(' ')) > 600:
        users_very_long.append(user)

In [16]:
print(len(users_very_long), users_very_long)

0 []


In [17]:
print(zeroshot_dict['102'])

You are an expert fashion product recommender. You are provided with a user's list of recent products and their descriptions that the user purchases and whether the user liked it or disliked it. Please go through the list in order -
Disliked Bellini DermaPad Anti-Wrinkle Reusable Chest Pad (1 Pad) Description: 

The product is a chest pad that claims to reduce wrinkles and plump the skin. It works well for a short time, but may cause irritation and does not stay on throughout the night.
Liked USonline911 Genuine Natural Hsiuyen Jade GuaSha Scraping Massage Tools for SPA Acupuncture Therapy Trigger Point Treatment Brand: USonline911 Price: $9.99 Description: 

The product is a facial massage tool, with a smooth texture and pretty color. It is great for massages and helps with wrinkles by plumping up the skin, releasing collagen build up, and giving a glow. It is a good price for home use. However, it does not meet
Liked Dalin 3D Nail Art Tips Stickers False Flower Nail Design Manicure D

In [16]:
# for key in users_very_long:
#     zeroshot_dict.pop(key, None)
# print(len(zeroshot_dict))

933


In [18]:
zeroshot_test_path = './ctr_zeroshot_dataset/zeroshot_valid.json'
with open(zeroshot_test_path, 'w+') as f:
    json.dump(zeroshot_dict, f)

In [19]:
ratings_df[ratings_df['user'] == int('0')]

,item,user,rating,timestamp
0,0,0,4.0,1155081600
1,1,0,5.0,1268697600
2,2,0,5.0,1404604800
3,3,0,5.0,1404604800
4,4,0,5.0,1404604800
5,5,0,5.0,1404604800
6,5,0,5.0,1404604800
7,6,0,5.0,1461801600


In [20]:
test_prompt = zeroshot_dict['34']
print(test_prompt)
print("-"*100)
response = openai.completions.create(
    # model = "gpt-3.5-turbo-0125",
    model="gpt-3.5-turbo-instruct",
    prompt = test_prompt,
    temperature=0.3,
    max_tokens=256,
    # top_p=0.3,
    # frequency_penalty=0.5,
    # presence_penalty=0.5
)
print(response.choices[0].text)

You are an expert fashion product recommender. You are provided with a user's list of recent products and their descriptions that the user purchases and whether the user liked it or disliked it. Please go through the list in order -
Liked Citre Shine Moisture Burst Shampoo - 16 fl oz Brand: Citre Shine Price: $23.00 Description: 

The product is a shampoo that helps reduce psoriasis irritation and thinning hair, while also moisturizing and strengthening all hair types. However, it may not be easily found in local stores.
Liked Bath &amp; Body Works Ile De Tahiti Moana Coconut Vanille Moana Body Wash with Tamanoi 8.5 oz Brand: Bath & Body Works Description: 

The product is a high-quality, good smelling cologne for men. It comes in various scents and is infused with pheromones. However, the scent may not last long after showering.

The product is a high-quality, good smelling cologne for
Liked Bath &amp; Body Works Ile De Tahiti Moana Coconut Vanille Moana Body Wash with Tamanoi 8.5 oz 